# Building a text classification model on the Amazon Reviews dataset
1. Inspect and process data with pandas and nltk
2. Store engineered features in Amazon SageMaker Feature Store (offline and online)
3. Build a dataset from the offline feature store with Amazon Athena
4. Train a classification model with Amazon SageMaker and BlazingText
5. Predict with features stored in the online store
6. Clean up

# 1 - Inspect and process data

In [ ]:
%%sh
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz /tmp

In [ ]:
import pandas as pd
import numpy as np
import time
from time import gmtime, strftime, sleep

In [ ]:
data = pd.read_csv('/tmp/amazon_reviews_us_Camera_v1_00.tsv.gz', sep='\t', 
                   compression='gzip', error_bad_lines=False, dtype='str')
data.dropna(inplace=True)

In [ ]:
print(data.shape)
print(data.columns)

In [ ]:
data.head()

In [ ]:
data = data[:100000]

In [ ]:
data['review_body'] = data['review_headline'] + ' ' + data['review_body']

In [ ]:
data.head()

In [ ]:
data = data[['review_id', 'product_id', 'star_rating', 'review_body']]

### Map 1-5 star ratings to negative/neutral/positive

In [ ]:
data.star_rating.unique()

In [ ]:
%%time
data['label'] = data.star_rating.map({
    '1': '__label__negative__',
    '2': '__label__negative__',
    '3': '__label__neutral__',
    '4': '__label__positive__',
    '5': '__label__positive__'})

In [ ]:
data = data.drop(['star_rating'], axis=1)

In [ ]:
data.head()

### Tokenize reviews

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
%%time
data['review_body'] = data['review_body'].apply(nltk.word_tokenize)

In [ ]:
%%time
data['review_body'] = data.apply(lambda row: " ".join(row['review_body']).lower(), axis=1)

In [ ]:
data.head()

# 2 - Create Feature Group and load data

In [ ]:
import boto3, sagemaker 
from sagemaker.session import Session

sagemaker_session = Session()
region = sagemaker_session.boto_region_name

boto_session = boto3.Session(region_name=region)
role = sagemaker.get_execution_role()

default_bucket = sagemaker_session.default_bucket()
prefix = 'amazon-reviews-featurestore'

sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

### Define the feature group name, and the name of the column for record identifiers

In [ ]:
from sagemaker.feature_store.feature_group import FeatureGroup

feature_group_name = 'amazon-reviews-feature-group-' + strftime('%d-%H-%M-%S', gmtime())
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=feature_store_session)

# The name of the column storing a unique record id (e.g. primary key)
record_identifier_feature_name = 'review_id'

### Add a column to store feature timestamps

In [ ]:
event_time_feature_name = 'event_time'
current_time_sec = int(round(time.time()))

# event_time is the name of the new column. A bit confusing!
data = data.assign(event_time=current_time_sec)

In [ ]:
# Make sure that timestamps are correctly set
# NaN timestamps will cause ingestion errors

data[data.isna().any(axis=1)]

In [ ]:
data.head()

### Infer feature definitions from the pandas dataframe

In [ ]:
data['review_id']     = data['review_id'].astype('str').astype('string')
data['product_id']    = data['product_id'].astype('str').astype('string')
data['review_body']   = data['review_body'].astype('str').astype('string')
data['label']         = data['label'].astype('str').astype('string')

data['event_time']    = data['event_time'].astype('float64')
# We could also use the UNIX date/time format, and we'd set the type to string

In [ ]:
feature_group.load_feature_definitions(data_frame=data)

### Create the feature group

In [ ]:
feature_group.create(
    s3_uri='s3://{}/{}'.format(default_bucket, prefix),
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=sagemaker.get_execution_role(),
    enable_online_store=True
)

In [ ]:
feature_group.describe().get("FeatureGroupStatus")

### Ingest features into our feature group, directly from the pandas dataframe

In [ ]:
feature_group.ingest(data_frame=data, max_workers=4, wait=True)

# 3 - Use Amazon Athena to build a training dataset

In [ ]:
feature_group_query = feature_group.athena_query()
feature_group_table = feature_group_query.table_name

print(feature_group_table)

### Build and run SQL query

In [ ]:
# BlazingText expects labels at the front

query_string = 'SELECT label,review_body FROM "'+feature_group_table+'";'

print(query_string)

In [ ]:
dataset = pd.DataFrame()
feature_group_query.run(query_string=query_string, output_location='s3://'+default_bucket+'/query_results/')
feature_group_query.wait()

In [ ]:
dataset = feature_group_query.as_dataframe()
dataset.head()

### Split dataset for training and validation, and save it to text files

In [ ]:
from sklearn.model_selection import train_test_split

training, validation = train_test_split(dataset, test_size=0.1)

In [ ]:
print(training.shape)
print(validation.shape)

In [ ]:
np.savetxt('/tmp/training.txt', training.values, fmt='%s')
np.savetxt('/tmp/validation.txt', validation.values, fmt='%s')

In [ ]:
!head -5 /tmp/training.txt

# 4 - Train a classification model on Amazon SageMaker with the BlazingText algorithm

In [ ]:
prefix = 'blazing-text-amazon-reviews'

s3_train_path = sagemaker_session.upload_data(path='/tmp/training.txt', bucket=default_bucket, key_prefix=prefix+'/input/train')
s3_val_path = sagemaker_session.upload_data(path='/tmp/validation.txt', bucket=default_bucket, key_prefix=prefix+'/input/validation')
s3_output = 's3://{}/{}/output/'.format(default_bucket, prefix)

print(s3_train_path)
print(s3_val_path)
print(s3_output)

In [ ]:
from sagemaker import image_uris

container = image_uris.retrieve('blazingtext', region)
print(container)

In [ ]:
bt = sagemaker.estimator.Estimator(container,
                                   role, 
                                   instance_count=1, 
                                   instance_type='ml.c5.2xlarge',
                                   output_path=s3_output)

In [ ]:
bt.set_hyperparameters(mode='supervised')

In [ ]:
train_data = sagemaker.TrainingInput(s3_train_path, 
                      distribution='FullyReplicated', 
                      content_type='text/plain',
                      s3_data_type='S3Prefix')

validation_data = sagemaker.TrainingInput(s3_val_path,
                      distribution='FullyReplicated', 
                      content_type='text/plain', 
                      s3_data_type='S3Prefix')

s3_channels = {'train': train_data, 'validation': validation_data}

In [ ]:
bt.fit(inputs=s3_channels)

In [ ]:
bt_predictor = bt.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

# 5 - Read a single record from the online store, and predict

In [ ]:
record_identifier_value = 'R273DCA6Y0H9V7'

response = featurestore_runtime.get_record(
    FeatureGroupName=feature_group_name, 
    RecordIdentifierValueAsString=record_identifier_value)

In [ ]:
import json
import pprint

pprint.pprint(response)

In [ ]:
def get_feature_value(record, feature_name):
    return str(list(filter(lambda r: r['FeatureName'] == feature_name, record))[0]['ValueAsString'])

In [ ]:
review_text = get_feature_value(response['Record'], 'review_body')

print(review_text)

In [ ]:
instances = []
instances.append(review_text)
print(instances)

In [ ]:
payload = {"instances" : instances, "configuration": {"k": 3}}

bt_predictor.serializer   = sagemaker.serializers.JSONSerializer()
bt_predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

response = bt_predictor.predict(payload)

pprint.pprint(response)

# 5 - Clean up

In [ ]:
bt_predictor.delete_endpoint()

In [ ]:
feature_group.delete()

In [ ]:
# How to remove old feature groups
old_feature_group_name = 'amazon-reviews-feature-group-19-10-47-35'
old_feature_group = FeatureGroup(name=old_feature_group_name, sagemaker_session=feature_store_session)
old_feature_group.delete()